# Last update: Feb 24 2022 by ivanp@us.ibm.com

In [1]:
#pip3 install pandas
import pandas as pd

from py2neo import Node
from py2neo import Relationship
from py2neo import Transaction
from py2neo.database import Graph

In [2]:
#Load ibm-blogs.csv

blogs_df = pd.read_csv("ibm-blogs.csv")
print("Number of rows in ibm-blogs file: ", blogs_df.shape[0])
blogs_df.head()

Number of rows in ibm-blogs file:  21


,id,title,link,date
0,1001,Cloud at the Edge,https://www.ibm.com/cloud/blog/cloud-at-the-edge,2/26/19
1,1002,Rounding out the Edges,https://www.ibm.com/cloud/blog/rounding-out-th...,5/7/19
2,1003,Architecting at the Edge,https://www.ibm.com/cloud/blog/architecting-at...,10/21/19
3,1004,DevOps at the Edge,https://www.ibm.com/cloud/blog/devops-at-the-edge,12/3/19
4,1005,Policies at the Edge,https://www.ibm.com/cloud/blog/policies-at-the...,1/22/20


In [4]:
id_df = blogs_df.set_index(blogs_df["id"])
id_df.drop(['id'], axis=1, inplace=True)
id_df.head()

,title,link,date
id,,,
1001,Cloud at the Edge,https://www.ibm.com/cloud/blog/cloud-at-the-edge,2/26/19
1002,Rounding out the Edges,https://www.ibm.com/cloud/blog/rounding-out-th...,5/7/19
1003,Architecting at the Edge,https://www.ibm.com/cloud/blog/architecting-at...,10/21/19
1004,DevOps at the Edge,https://www.ibm.com/cloud/blog/devops-at-the-edge,12/3/19
1005,Policies at the Edge,https://www.ibm.com/cloud/blog/policies-at-the...,1/22/20


In [5]:
nodes = sorted(list(blogs_df.id.unique()) )
len(nodes)
print(nodes)

[1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021]


In [6]:
def _node(an_id:str) -> Node:
    return Node(":Article" , title=id_df.at[an_id,'title'],  article_id=str(an_id),
                link=id_df.at[an_id,'link'], date=id_df.at[an_id,'date']
                )

d_nodes = dict((key, _node(key)) for (key) in nodes)
print(d_nodes)

{1001: Node(':Article', article_id='1001', date='2/26/19', link='https://www.ibm.com/cloud/blog/cloud-at-the-edge', title='Cloud at the Edge'), 1002: Node(':Article', article_id='1002', date='5/7/19', link='https://www.ibm.com/cloud/blog/rounding-out-the-edges', title='Rounding out the Edges'), 1003: Node(':Article', article_id='1003', date='10/21/19', link='https://www.ibm.com/cloud/blog/architecting-at-the-edge', title='Architecting at the Edge'), 1004: Node(':Article', article_id='1004', date='12/3/19', link='https://www.ibm.com/cloud/blog/devops-at-the-edge', title='DevOps at the Edge'), 1005: Node(':Article', article_id='1005', date='1/22/20', link='https://www.ibm.com/cloud/blog/policies-at-the-edge', title='Policies at the Edge'), 1006: Node(':Article', article_id='1006', date='5/12/20', link='https://www.ibm.com/cloud/blog/security-at-the-edge', title='Security at the Edge'), 1007: Node(':Article', article_id='1007', date='6/8/20', link='https://www.ibm.com/cloud/blog/analytics

In [7]:
url = "bolt://localhost:7687"
db = "talk2"
pwd = "{PASSWORD}"

neog = Graph(url, db, auth=('neo4j', pwd))

In [8]:
tx = neog.begin()
neog.delete_all()
neog.commit(tx) 

In [9]:
def countNodes():
    query = "MATCH (n) RETURN COUNT(n)"
    result = neog.query(query)
    print(result)

In [13]:
def createNodes(d_nodes:dict):
    for node in d_nodes.values():
        tx.create(node)
    print ("Neo4j nodes created")

In [10]:
# Creates a triple series with item1 - item2 - item3 as input

def create_tpl(itemOne, itemTwo, itemThree):
    tpl = []
    tpl.append(itemOne)
    tpl.append(itemTwo)
    tpl.append(itemThree)
    return tpl

In [18]:
def get_unique(column):
    """
    Get unique elements from a Pandas DF column
    Arguments:
        column: a DF column
    Returns:
        A node list with sorted, 
        unique elements of the DF column
    """
    nodes = ()
    nodes = sorted( list(column.unique() ) )
    return nodes 

In [11]:
countNodes()

 COUNT(n) 
----------
        0 



In [14]:
#Create article nodes
tx = neog.begin()
createNodes(d_nodes)
neog.commit(tx) 

Neo4j nodes created


In [15]:
countNodes()

 COUNT(n) 
----------
       21 



In [16]:
c_df = pd.read_csv ('final-categories.csv')
print(c_df)

                                                  URL  \
0    https://www.ibm.com/cloud/blog/cloud-at-the-edge   
1    https://www.ibm.com/cloud/blog/cloud-at-the-edge   
2    https://www.ibm.com/cloud/blog/cloud-at-the-edge   
3   https://www.ibm.com/cloud/blog/rounding-out-th...   
4   https://www.ibm.com/cloud/blog/rounding-out-th...   
..                                                ...   
56  https://www.ibm.com/cloud/blog/truck-tracker-a...   
57  https://www.ibm.com/cloud/blog/truck-tracker-a...   
58  https://www.ibm.com/cloud/blog/optical-charact...   
59  https://www.ibm.com/cloud/blog/optical-charact...   
60  https://www.ibm.com/cloud/blog/optical-charact...   

                                                Label     Score  
0                           /technology and computing  0.869217  
1       /technology and computing/internet technology  0.823215  
2   /technology and computing/networking/network m...  0.808410  
3                  /technology and computing/hardwa

In [17]:
rows = []
for _, row in c_df.iterrows():
    #print(row['Label'])
    cats = row['Label'].split('/')
    cats = list(filter(None, cats))
    
    for c in cats:
        tpl = create_tpl(row['URL'], c, row['Score'])
        rows.append(tpl)

#print(rows)
cat_df = pd.DataFrame(rows)
print(cat_df.shape)
print(cat_df.head())

(155, 3)
                                                    0  \
0    https://www.ibm.com/cloud/blog/cloud-at-the-edge   
1    https://www.ibm.com/cloud/blog/cloud-at-the-edge   
2    https://www.ibm.com/cloud/blog/cloud-at-the-edge   
3    https://www.ibm.com/cloud/blog/cloud-at-the-edge   
4    https://www.ibm.com/cloud/blog/cloud-at-the-edge   
5    https://www.ibm.com/cloud/blog/cloud-at-the-edge   
6   https://www.ibm.com/cloud/blog/rounding-out-th...   
7   https://www.ibm.com/cloud/blog/rounding-out-th...   
8   https://www.ibm.com/cloud/blog/rounding-out-th...   
9   https://www.ibm.com/cloud/blog/rounding-out-th...   
10  https://www.ibm.com/cloud/blog/rounding-out-th...   
11  https://www.ibm.com/cloud/blog/rounding-out-th...   
12  https://www.ibm.com/cloud/blog/rounding-out-th...   
13  https://www.ibm.com/cloud/blog/architecting-at...   
14  https://www.ibm.com/cloud/blog/architecting-at...   
15  https://www.ibm.com/cloud/blog/architecting-at...   
16  https://www.ibm.co

In [20]:
#create node list from a tuple
#tuple:ordered collection of objects

#category node list
c_nodes = get_unique(cat_df[1])
print(c_nodes)
#type(c_nodes)

['automation', 'business and industrial', 'business operations', 'business process', 'business software', 'computer', 'computer components', 'computer networking', 'computer peripherals', 'computer security', 'databases', 'email', 'hardware', 'internet technology', 'management', 'network monitoring and management', 'network security', 'networking', 'operating systems', 'servers', 'software', 'technology and computing', 'unix', 'wireless technology']


In [21]:
#Create a dictionary of categorry ID nodes with p_nodes list
d_nodes =()
def _node(category:str) -> Node:
    return Node(":Category",  name=category , label = category
    )

d_nodes = dict((key, _node(key)) for (key) in c_nodes) 
print("Dictionary size: " + str(len(d_nodes)) )
print(d_nodes)

Dictionary size: 24
{'automation': Node(':Category', label='automation', name='automation'), 'business and industrial': Node(':Category', label='business and industrial', name='business and industrial'), 'business operations': Node(':Category', label='business operations', name='business operations'), 'business process': Node(':Category', label='business process', name='business process'), 'business software': Node(':Category', label='business software', name='business software'), 'computer': Node(':Category', label='computer', name='computer'), 'computer components': Node(':Category', label='computer components', name='computer components'), 'computer networking': Node(':Category', label='computer networking', name='computer networking'), 'computer peripherals': Node(':Category', label='computer peripherals', name='computer peripherals'), 'computer security': Node(':Category', label='computer security', name='computer security'), 'databases': Node(':Category', label='databases', name=

In [22]:
#Create category nodes
tx = neog.begin()
createNodes(d_nodes)
neog.commit(tx) 

Neo4j nodes created


In [23]:
countNodes()

 COUNT(n) 
----------
       45 



In [ ]:
#Create URL -has-a-> Category relationships

In [24]:
rows=[]

for _, row in c_df.iterrows():
    #print(row['URL'])
    #print(row['Label'])
    cats = row['Label'].split('/')
    cats = list(filter(None, cats))
    #print(cats)
    #print(cats[-1])
    tpl = create_tpl(row['URL'], 'HAS-A', cats[-1])
    rows.append(tpl)


print(rows)
has_df = pd.DataFrame(rows)
print(has_df.shape)
print(has_df.head())

[['https://www.ibm.com/cloud/blog/cloud-at-the-edge', 'HAS-A', 'technology and computing'], ['https://www.ibm.com/cloud/blog/cloud-at-the-edge', 'HAS-A', 'internet technology'], ['https://www.ibm.com/cloud/blog/cloud-at-the-edge', 'HAS-A', 'network monitoring and management'], ['https://www.ibm.com/cloud/blog/rounding-out-the-edges', 'HAS-A', 'hardware'], ['https://www.ibm.com/cloud/blog/rounding-out-the-edges', 'HAS-A', 'operating systems'], ['https://www.ibm.com/cloud/blog/rounding-out-the-edges', 'HAS-A', 'network monitoring and management'], ['https://www.ibm.com/cloud/blog/architecting-at-the-edge', 'HAS-A', 'hardware'], ['https://www.ibm.com/cloud/blog/architecting-at-the-edge', 'HAS-A', 'operating systems'], ['https://www.ibm.com/cloud/blog/architecting-at-the-edge', 'HAS-A', 'computer'], ['https://www.ibm.com/cloud/blog/devops-at-the-edge', 'HAS-A', 'hardware'], ['https://www.ibm.com/cloud/blog/devops-at-the-edge', 'HAS-A', 'operating systems'], ['https://www.ibm.com/cloud/blog

In [25]:
has_df.rename(columns={0: 'URL', 1: 'Relationship', 2: 'Category'}, inplace=True )
print(has_df.head())
print(has_df.shape)

                                                 URL Relationship  \
0   https://www.ibm.com/cloud/blog/cloud-at-the-edge        HAS-A   
1   https://www.ibm.com/cloud/blog/cloud-at-the-edge        HAS-A   
2   https://www.ibm.com/cloud/blog/cloud-at-the-edge        HAS-A   
3  https://www.ibm.com/cloud/blog/rounding-out-th...        HAS-A   
4  https://www.ibm.com/cloud/blog/rounding-out-th...        HAS-A   

                            Category  
0           technology and computing  
1                internet technology  
2  network monitoring and management  
3                           hardware  
4                  operating systems  
(61, 3)


In [26]:
#If we need to remove duplicates


#category1_df = has_df.drop_duplicates(
# subset = ['URL', 'Relationship', 'Category'],
#  keep = 'last').reset_index(drop = True)
  
# print latest dataframe
#print(category1_df.head())
#print(category1_df.shape)

                                                 URL Relationship  \
0   https://www.ibm.com/cloud/blog/cloud-at-the-edge        HAS-A   
1   https://www.ibm.com/cloud/blog/cloud-at-the-edge        HAS-A   
2   https://www.ibm.com/cloud/blog/cloud-at-the-edge        HAS-A   
3  https://www.ibm.com/cloud/blog/rounding-out-th...        HAS-A   
4  https://www.ibm.com/cloud/blog/rounding-out-th...        HAS-A   

                            Category  
0           technology and computing  
1                internet technology  
2  network monitoring and management  
3                           hardware  
4                  operating systems  
(61, 3)


In [27]:
tx = neog.begin()

for _, row in has_df.iterrows():
    node = "MATCH(a:`:Article`) WHERE a.link='" + row[0] + "' return a"
    a_node  =  neog.evaluate(node)
    node = "MATCH(c:`:Category`) WHERE c.label='" + row[2] + "' return c"
    c_node  =  neog.evaluate(node)
    tx.create(Relationship(a_node, row[1], c_node))

print("URL-relationship-Category completed")

neog.commit(tx)

URL-relationship-Category completed


In [ ]:
#Create category-has-a->category relationships

In [28]:
rows=[]

for _, row in c_df.iterrows():
    #print(row['URL'])
    #print(row['Label'])
    cats = row['Label'].split('/')
    cats = list(filter(None, cats))
    #print('\nrow: ', cats)
    #print(type(cats))
    for c in cats:
        if (len(cats) >= 2):
            tpl = create_tpl(cats[0], 'HAS-A', cats[1])
            rows.append(tpl)
            cats.pop(0)

ontology_df = pd.DataFrame(rows)
print(ontology_df.shape)
print(ontology_df.head())

(88, 3)
                          0      1                                  2
0  technology and computing  HAS-A                internet technology
1  technology and computing  HAS-A                         networking
2                networking  HAS-A  network monitoring and management
3  technology and computing  HAS-A                           hardware
4  technology and computing  HAS-A                  operating systems


In [29]:

ontology_df.rename(columns={0: 'Category', 1: 'Relationship', 2: 'Category'}, inplace=True )
print(ontology_df.head())
print(ontology_df.shape)

                   Category Relationship                           Category
0  technology and computing        HAS-A                internet technology
1  technology and computing        HAS-A                         networking
2                networking        HAS-A  network monitoring and management
3  technology and computing        HAS-A                           hardware
4  technology and computing        HAS-A                  operating systems
(88, 3)


In [30]:
new_df = ontology_df.drop_duplicates(
  subset = ['Category', 'Relationship', 'Category'],
  keep = 'last').reset_index(drop = True)
  
# print latest dataframe
print(new_df.head())
print(new_df.shape)

                   Category Relationship              Category
0                  hardware        HAS-A  computer peripherals
1  technology and computing        HAS-A     computer security
2         computer security        HAS-A      network security
3   business and industrial        HAS-A            automation
4  technology and computing        HAS-A            networking
(19, 3)


In [34]:
tx = neog.begin()

for _, row in new_df.iterrows():
    node = "MATCH(c:`:Category`) WHERE c.label='" + row[0] + "' return c"
    a_node  =  neog.evaluate(node)
    node = "MATCH(c:`:Category`) WHERE c.label='" + row[2] + "' return c"
    b_node  =  neog.evaluate(node)
    tx.create(Relationship(a_node, row[1], b_node))

print("Category-relationship-Category completed")

neog.commit(tx)

Category-relationship-Category completed


In [35]:
countNodes()

 COUNT(n) 
----------
       45 



In [36]:
#open GDSL playground
#pagerank
#lovain